# Stochastic Series Expansion
## Mathematical Formalism
In quantum statistical mechanics, we are interested in calculating thermodynamical properties of quantum systems. To do so we often need to calculate an expectation value of some operator, which is the trace, $\langle A \rangle = \text{Tr}{Ae^{-\beta H}}$ and the partition function $Z = \text{Tr} e^{-\beta H}$. In general, this is difficult to calculate, in fact intractable for scaling system sizes. This problem poses two main problems, firstly, the exponential of a non-diagonal operator $H$ is difficult to calculate, and non-trivial systems include non-commuting and therefore off-diagonal terms, secondly the sum in the Trace can be very large, and secondly. SSE solves the first problem by taking the Taylor expansion of the exponential, and replacing the trace with a sum over an appropriate basis. 

Since the Hamiltonian operator is still non-diagonal, we first decompose the operator into diagonal parts, $H = -\sum_{a,b} H_{a,b}$ doubly indexed with $a$ being the class, diagonal or off-diagonal, and $b$ being the index on which the particular component of the Hamiltonian acts on. Powers of the Hamiltonian are then the sums of all possible products of the Hamiltonian components

$$(-H)^n = \sum_{H_{a,b}} \prod^n_p H_{a(p), b(p)}$$

The goal of SSE is to sample the major contributors to this sum to approximate this expression, which gives us a suitable expression for operator expectation values and the partition function.

## SSE Algorithm and Implementation for XYZ Model
The Hamiltonian for the model is given by 

$$ H = \sum J_x \sigma_i^x\sigma_j^x + \sum J_y \sigma_i^y\sigma_j^y + \sum J_z \sigma_i^z\sigma_j^z $$

The off-diagonal terms are obviously the terms with the Pauli $y$ and $x$ operators, diagonal terms being the Pauli $z$. To sample the operator configurations, we store a list of operators. $H_{a=0,b=0}$ denoting the identity or null operator. We then proceed to describe 3 typical updating schemes. 

### Diagonal Updates
To add, change, or remove a diagonal operator is fairly unrestricted. Diagonal operators don't flip spins, so they do not have long range effects in the spin states when they are propogated in time. Whereas off-diagonal updates are non-local and potentially produce invalid states, diagonal updates are local and well behaved. We seek only to maintain detailed balance in our diagonal update step. 

Any identity operator in our list, when replaced with a diagonal operator, remains a valid configuration, and removing a diagonal operator for a identity operator is permitted as well.

### Off Diagonal Pair Update
As mentioned previously, an off-diagonal operator such as $\sigma^x$ would flip the spin that it acts on. In order to include operators, spins that propogate through the operator list must eventually flip back to the proper ending configuration. One potential updating scheme is to find pairs adjacent pairs of identical operators, separated only by time, and flip both. 

### Cluster Updates
Cluster updates can be implemented with an operator-loop algorithm. It is clear that the operator list, each operator corresponding to a configuration of states as the operators are applied, containes redundant information. For example, Take a spin immediately after being acted on by an operator, and immediately before the next operator is applied. The spin configuration is necessarily the same regardless of the operator, and must be stored twice. Instead of storing states in time slices, we can switch at this point to a new representation. Each operator can be thought of as a "vertex" `V`in an undirected graph, `G`, linked to 4 "legs" or edges `E`. A series of identical spin configurations could be represented by a single edge. 

With this representation of the configuration, we can introduce the operator-loop procedure. The idea is to create a cluster of linked spins such that when they are all flipped, the new configuration remains valid. The intuition is simple, an operator acting on two spins has 2 incoming legs and 2 outgoing legs. If one of those is flipped, it is obvious that at least one other must also be flipped. If an incoming leg is flipped, a suggestion could be that the corresponding outgoing leg is flipped as well, traversing across the operator. But notice that here, if the leg on the same side is flipped, e.g. an incoming pair or outgoing pair, then the spin states crossing the operator are now oriented differently from before, so the operator changes between diagonal and off-diagonal operators. 

With this, we can propose a procedure for updating spins. Build all the clusters, and flip them with probability 1/2. Before the process of building a loop, we can decide whether this particular loop will be flipped, and record this as the loop is being constructed. In one update, one particular vertex can only be traversed once, since loops are closed, so we can mark a loop as visited and ensure that we never try to flip the same cluster twice. Example pseudocode can be given as follows:


```
OperatorLoopUpdate(G(V,E)) :
    select a starting vertex v0
    mark v0 as visited
    let vn = vertexlist[v0]
    while (not vn = v0)
        mark vn as visited
        let vn+1
```

## Simple Implementation
### Structures
We start by introducing some types that we will use to define our model. We will need a struct that represents the current spin configuration, which we will call Lattice. We also want a method of keeping track of our operator list, which is a structure that will identify the type of operator, as well as a unique index that identifies which bonds the operator acts on.

In [1]:
struct Lattice{D, T}
    dims::Dims{D}
    sites::Array{CartesianIndex{D}}
    neighbours::Array{CartesianIndex{D}, T}
    spins::Array{Int64, D}
    function Lattice(N::Int64)
        dims = (N,N)
        z = length(dims)
        sites = CartesianIndices(dims)
        neighbours = cat(circshift(sites, (-1,  0)),
                         circshift(sites, ( 1,  0)),
                         circshift(sites, ( 0, -1)),
                         circshift(sites, ( 0,  1)),
                         dims = z + 1)
        spins = rand(-1:2:1, dims)
        new{z, z+1}(dims, sites, neighbours, spins)
    end
end

mutable struct Operator
    op::Symbol
    bond::Int64
    # TODO: change struct wrapping to avoid mutable
end

struct SSE{D}
    N::Int64             # System size Lx=Ly=N
    Nb::Int64            # # of bonds, in 1D N=Nb but doesn't generalize to higher dimensions
    M::Base.RefValue{Int64}   # Taylor Expansion cutoff point (can we do better than ref/refvalue?)
    n::Base.RefValue{Int64}             # # of operators in list
    β::Float64           # thermodynamic beta
    ops::Array{Operator}
    lattice::Lattice
    vertexlist::Array{Int64}
    function SSE(N::Int64, β::Float64)
        lattice = Lattice(N)
        Nb = 2*N*N
        M = Int64(round(Nb*1.5*β))
        z = N*N
        ops = [Operator(:null,0) for i=1:M]
        vertexlist = fill(-1, M*4)
        new{z}(N, Nb, Ref(M), Ref(0), β, ops, lattice, vertexlist)
    end
end

s = SSE(4,10.0);

We must also introduce the indexing scheme for key values. `CartesianIndex` lets us index the sites as a matrix $i,j$ and also as numbered items from $1$ to $N \times N$. We will index our bonds between nearest neighbours from $1$ to $N_b$, where a bond between sites $[1,2] = 1$, $[2,3] = 2$, etc. horizontal bonds first. The first vertical bond $[1, N+1] = N^2+1$. If the bond is horizontal, an "x" bond, getting the sites that the bond acts on is simple, $x = [x, x+1]$. If the bond is vertical, a "y" bond, the sites it acts on are given by $y = [y-N^2, y-N^2-N]$. We store this bond as a tuple of the index as well as the sites they act on for convenience. This allows us to uniquely index all the possible bonds in a lattice.

In [2]:
function getBonds(index::Int64, sites::Array{CartesianIndex{2}, 2})
    (x,y) = size(sites)
    if index > x*y
        return (index, sites[index-x^2], sites[mod1(index-x^2+x,x*x)])
    else
        return (index, sites[index], sites[x*div(index-1,x)+mod1(index+1,x)])
    end
end

getBonds (generic function with 1 method)

With these, we can begin to implement our updating schemes, beginning with the diagonal update. The diagonal update begins with iterating over every operator in the list, if the operator is empty, we have a chance to insert a new diagonal operator, if a diagonal operator exists, we have a chance to remove it. The probabilities of such transformations must maintain detailed balance, specifically

$$\begin{align*}
    P(x \rightarrow x^\prime)W(x) = P(x^\prime \rightarrow x)W(x^\prime)
\end{align*}$$

We have separated the overall transition probability into the probability of selecting this transition and the probability of accepting this transition. The weight of a particular configuration is simple to calculate, if expansions up to order $M$ are sampled, then the weight is a function of the number of non-empty operators $n$ and $\beta$

$$\begin{align*}
    W(x) = \left(\frac{\beta}{2}\right)^n \frac{(M-n)!}{M!}
\end{align*}$$

Notice however, if the only change we make is a single `:diag` operator to a `:null` or vice versa, then $n = n \pm 1$. The probability of accepting an insertion is weighted by 

$$\begin{align*}
    \frac{\beta /2}{M-n}
\end{align*}$$

The probability of inserting vs. removing a bond is the same for a particular time slice, but the insertion has an additional option of one of $N_b$ bond indices. Therefore, the probability of inserting an operator is 

$$\begin{align*}
    P_{insert} = \frac{N_b \beta /2}{M-n}
\end{align*}$$

And the probability of removing is

$$\begin{align*}
    P_{remove} = \frac{M-n+1}{N_b \beta /2}
\end{align*}$$

In [3]:
function diagonalUpdate!(state::SSE)
    for i in state.ops
        if i.op == :diag
            P = state.Nb*state.β/2/(state.M.x-state.n.x)
            if rand() < P
                i.op = :null
                i.bond = 0
                state.n.x -= 1
            end
        elseif i.op == :null
            index = rand(1:state.Nb)
            bond = getBonds(index, state.lattice.sites)
            if state.lattice.spins[bond[2]] != state.lattice.spins[bond[3]]
                P = 2*(state.M.x-state.n.x+1)/state.Nb/state.β
                if rand() < P
                    i.op = :diag
                    i.bond = index
                    state.n.x += 1
                end
            end
        elseif i.op == :offdiag
            # propogate spins
            bond = getBonds(i.bond, state.lattice.sites)
            state.lattice.spins[bond[2]] *= -1
            state.lattice.spins[bond[3]] *= -1
        end
    end
end


diagonalUpdate! (generic function with 1 method)

If we encounter a null or a diagonal operator, we know the monte carlo acceptance probabilities and update the operator list accordingly. However, we need to also do something when we have an off-diagonal operators as they appear, since they flip the spin configuration.

After a diagonal update, we proceed to build clusters and flip them for the off-diagonal update. To do so, we define a new structure that keeps track of our linked vertex list and clusters. Each leg on a vertex is assigned a leg, indexed $1-4$, note that Julia's convention indexes lists starting at 1.

```
    1 2
    |—|
    3 4
```

Before we can identify any clusters within the vertex graph, we need to construct it. To do so, we iterate over the operator list, keeping track of the index of the last leg to act on an operator. When we see a new leg appear in that site, we connect them. Finally, the last operator is linked to the first to maintain periodic boundary conditions, so as we iterate over the operator list, we keep track of that as well. 

In [4]:
function buildVertexList(state::SSE)
    lastVertex = fill(-1,state.N,state.N)
    firstVertex = fill(-1,state.N,state.N)
    fill!(state.vertexlist, -1)
    for (i,op) in enumerate(state.ops)
        if op.op != :null
            v0 = 4*(i-1)+1
            b = getBonds(op.bond, state.lattice.sites)
            v1 = lastVertex[b[2]]
            v2 = lastVertex[b[3]]
            if v1 == -1
                firstVertex[b[2]] = v0
            else
                state.vertexlist[v0] = v1
                state.vertexlist[v1] = v0
            end
            if v2 == -1
                firstVertex[b[3]] = v0 + 1
            else
                state.vertexlist[v0+1] = v2
                state.vertexlist[v2] = v0+1
            end
            lastVertex[b[2]] = v0+2
            lastVertex[b[3]] = v0+3
        end
    end
    for site in 1:state.N*state.N
        f = firstVertex[site]
        l = lastVertex[site]
        if f != -1
            state.vertexlist[ f ] = l
            state.vertexlist[ l ] = f
        end
    end
end

buildVertexList (generic function with 1 method)

We can "enter" the vertex from any leg, but we must exit from the adjacent leg without crossing the operator. The following function returns the index of the next vertex.

In [5]:
function next(v::Int64)
    return xor(v-1,1)+1
end

next (generic function with 1 method)

The integer stored at `vertexlist[i]` is the index of the leg that `i` is linked to, so we can traverse a closed loop easily. The vertexlist also stores information about whether or not we have visited the node, and whether or not it should be flipped. 

In [6]:
function loopUpdate!(state::SSE)
    v0 = 1
    for (i,v) in enumerate(state.vertexlist)
        if v != -1
            v0 = i
            break
        end
    end
    flip = true # rand(Bool)
    v1 = next(v0)
    v1 = state.vertexlist[v1]
    while v1 != v0
        if flip
            if state.ops[div(v1-1,4)+1].op == :diag
                state.ops[div(v1-1,4)+1].op = :offdiag
            else
                state.ops[div(v1-1,4)+1].op = :diag
            end
        end
        v1 = next(v1)
        v1 = state.vertexlist[v1]
    end
    if flip
        if state.ops[div(v0-1,4)+1].op == :diag
            state.ops[div(v0-1,4)+1].op = :offdiag
        else
            state.ops[div(v0-1,4)+1].op = :diag
        end
    end
end


loopUpdate! (generic function with 1 method)

Putting it all together, we can write a single function that updates our state with the SSE algorithm

In [7]:
function MCSweep!(s::SSE)
    diagonalUpdate!(s)
    buildVertexList(s)
    loopUpdate!(s)
end

MCSweep! (generic function with 1 method)

After we perform a Monte Carlo sweep, we need to keep track of some variables so that we can easily measure observables later. The energy, for example, can be calculated as the expectation value

$$\begin{align*}
    E = -\frac{\langle n \rangle}{\beta}
\end{align*}$$

In [8]:
using Statistics

function measureObservables!(nsweeps::Int64, s::SSE)
    n = Array{Int64, 1}(undef, nsweeps)
    avgE = Array{Float64, 1}(undef, nsweeps)
    for i in 1:nsweeps
        MCSweep!(s)
        n[i] = s.n.x
        avgE[i] = -mean(n[1:i])/s.β
    end
    return avgE[nsweeps]
end

measureObservables! (generic function with 1 method)

In [9]:
measureObservables!(1000,s)

-26.964